In [1]:
import numpy as np
import pandas as pd
from main import run_di, run_ehd, experiments
from sim_env import FixedSimEnv
from objects import td

In [ ]:
pax, trips, events, wlogs = experiments()

In [2]:
env = FixedSimEnv()
unique_dates = env.link_times['date'].unique()
dates = np.random.choice(unique_dates, replace=False, size=3)

In [3]:
next_obs, rew, done, info = env.reset(hist_date=dates[0])
print(env.hist_date)

2022-11-29


In [29]:
env, info, n_steps, ref_veh_df, rts_info, best_switch = run_di(env, done)
print(td(env.time), n_steps)

while loop:  13%|█▎        | 1590/12000 [00:29<03:15, 53.25it/s]

92 81
34110.0 33690.0
92 81
34110.0 33690.0
0.7063422983699943 0.6980647275989049
1.0118578842961472
0 days 07:42:30 1590


In [12]:
from interlining import *

In [29]:
env, info, n_steps, upd_info = run_ehd(env, done, n_steps=n_steps, debug=True)
n_steps

----
Time 0 days 05:55:48
Expected: 0 days 05:56:30
Headways: 0 days 00:08:32 -- 0 days 00:09:00
Schedule: 0 days 05:56:30
0 days 05:55:48 < Departure < 0 days 06:01:30
Recommended 0 days 05:56:44 ---> 0 days 05:56:44


816